In [ ]:
# wide extend of jupyter
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
%load_ext autoreload
%autoreload 2
from IPython.core.display import Image, display

import os
import numpy as np
from sklearn.model_selection import train_test_split
from IPython.core.display import Image, display, HTML
import pandas as pd
from tqdm import tqdm
tqdm.pandas(desc='progress')
pd.set_option('max_colwidth',200)
import matplotlib.pyplot as plt

FILE_PATH = '/home/eiao/pythoncode/FaceReco'

# data loading

In [ ]:
label_file_path = os.path.join(FILE_PATH, 'datasets/data.csv')
data_df = pd.read_csv(label_file_path)
data_df['image'] = data_df['image'].apply(lambda x: os.path.join(FILE_PATH, 'dataset/image', x))
data_df = data_df.sample(frac=0.01, replace=True)
data_df

In [ ]:
from keras.callbacks import CSVLogger, ModelCheckpoint, EarlyStopping
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator

from models.cnn import mini_XCEPTION
from utils.datasets import DataManager
from utils.datasets import split_data
from utils.preprocessor import preprocess_input

# parameters
batch_size = 32
num_epochs = 10000
input_shape = (64, 64, 1)
validation_split = .2
verbose = 1
num_classes = 7
patience = 50
base_path = '../trained_models/emotion_models/'

# data generator
data_generator = ImageDataGenerator(
                        featurewise_center=False,
                        featurewise_std_normalization=False,
                        rotation_range=10,
                        width_shift_range=0.1,
                        height_shift_range=0.1,
                        zoom_range=.1,
                        horizontal_flip=True)

# model parameters/compilation
model = mini_XCEPTION(input_shape, num_classes)
model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()


datasets = ['fer2013']
for dataset_name in datasets:
    print('Training dataset:', dataset_name)

    # callbacks
    log_file_path = base_path + dataset_name + '_emotion_training.log'
    csv_logger = CSVLogger(log_file_path, append=False)
    early_stop = EarlyStopping('val_loss', patience=patience)
    reduce_lr = ReduceLROnPlateau('val_loss', factor=0.1,
                                  patience=int(patience/4), verbose=1)
    trained_models_path = base_path + dataset_name + '_mini_XCEPTION'
    model_names = trained_models_path + '.{epoch:02d}-{val_acc:.2f}.hdf5'
    model_checkpoint = ModelCheckpoint(model_names, 'val_loss', verbose=1,
                                                    save_best_only=True)
    callbacks = [model_checkpoint, csv_logger, early_stop, reduce_lr]

    # loading dataset
    data_loader = DataManager(dataset_name, image_size=input_shape[:2])
    faces, emotions = data_loader.get_data()
    faces = preprocess_input(faces)
    num_samples, num_classes = emotions.shape
    train_data, val_data = split_data(faces, emotions, validation_split)
    train_faces, train_emotions = train_data
    model.fit_generator(data_generator.flow(train_faces, train_emotions,
                                            batch_size),
                        steps_per_epoch=len(train_faces) / batch_size,
                        epochs=num_epochs, verbose=1, callbacks=callbacks,
                        validation_data=val_data)


In [ ]:

train_data,test_data,train_targets,test_targets = train_test_split(data_df['image'].tolist(),data_df['label'].tolist(),test_size=0.2)

# data loader
image_train_dataset = ImageDataset(train_data, train_targets, transform=image_train_transform)
image_test_dataset = ImageDataset(test_data, test_targets, transform=image_test_transform)

train_loader = torch.utils.data.DataLoader(image_train_dataset, batch_size=1, shuffle=True)
test_loader = torch.utils.data.DataLoader(image_test_dataset, batch_size=1, shuffle=False)
# train_loader = torch.utils.data.DataLoader(image_train_dataset, batch_size=32, shuffle=True, num_workers=8, pin_memory=True)
# test_loader = torch.utils.data.DataLoader(image_test_dataset, batch_size=32, shuffle=False, num_workers=8, pin_memory=True)

In [ ]:
len(train_targets)

# model building

In [ ]:
from models.resnext_101_64x4d import resnext_101_64x4d
from models.model import ResnextLogo
from torch import nn
from torch import optim

IMG_MODEL_PATH = os.path.join(FILE_PATH, 'trained_model')   # save trained model at this path

model = resnext_101_64x4d
model.load_state_dict(torch.load(os.path.join(FILE_PATH, 'models/resnext_101_64x4d.pth')))
model = ResnextLogo(model)

for param in model.parameters():
    param.require_grad = True
for param in list(model.children())[0].parameters():
    param.require_grad = False
# model.cuda()
model_name = '_'.join([model.__class__.__name__.lower(), '101', '64*4d', str(int(time.time()))])
print(model_name)

# model training

In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

model.train()

from utils.train import train, validate, adjust_learning_rate, save_checkpoint
epochs = 70
model_version = "_".join([model_name, 'small_test', 'Adam', 'lr: 0.001_0.01'])

# trian parameters:  
layer_resnext_train_parameters = list(model.children())[1].parameters()
layer_classifier_parameters = list(model.children())[2].parameters()

#choose lr: 
# optimizer = torch.optim.RMSprop([{"params":layerfc_parameters,"lr":0.01},{"params":layer4_parameters,"lr":0.001},{"params":layer3_parameters,"lr":1e-4},{"params":base_params,"lr":1e-4}], momentum=0.9, weight_decay=1e-5)
optimizer = torch.optim.Adam([{"params":layer_resnext_train_parameters,"lr":0.001},{"params":layer_classifier_parameters,"lr":0.01}])

criterion = nn.CrossEntropyLoss()
# criterion = nn.CrossEntropyLoss().cuda()

best_prec1 = 0
loss_epoch = []
metrics = list()

for epoch in range(1, epochs+1):
#     adjust_learning_rate(optimizer, epoch, base_lr)
    sys.stdout.flush()
    sys.stdout.write('-' * 20)
    sys.stdout.write('\n* Epoch {}\n'.format(epoch))

    # train for one epoch
    train_prec1, train_precn, train_loss = train(train_loader, model, criterion, optimizer, top_num=3)
    print()
    # evaluate on validation set
    val_prec1, val_precn, val_loss = validate(test_loader, model, criterion, top_num=3)

    sys.stdout.write('\n* Prec@1 {val_prec1:.3f} {now}\n'.format(val_prec1=val_prec1, now=datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')))
    sys.stdout.flush()

    metrics.append(dict(
        epoch=epoch,
        train_prec1 = train_prec1,
        train_precn = train_precn,
        train_loss = train_loss,
        val_prec1 = val_prec1,
        val_precn = val_precn,
        val_loss = val_loss
    ))

    is_best = val_prec1 > best_prec1
    best_prec1 = max(val_prec1, best_prec1)
    save_checkpoint({
            'epoch': epoch + 1,
            'arch': model_name,
            'state_dict': model.state_dict(),
            'best_prec1': best_prec1,
            'model_version': model_version,
            'version': model_version,
            'metrics': metrics,
        }, 'best' if is_best else 'normal', IMG_MODEL_PATH, model_name)

# model evalution

# Resnet101

In [ ]:
metrics_df = pd.DataFrame(metrics)
import matplotlib.pyplot as plt
metrics_df[['train_prec1', 'val_prec1']].plot(xlim=(0, len(metrics_df)), ylim=(0, 100))
metrics_df[['train_loss', 'val_loss']].plot(xlim=(0, len(metrics_df)))
plt.show()

In [ ]:
metrics_df

# Resnet50

In [ ]:
metrics_df = pd.DataFrame(metrics)
import matplotlib.pyplot as plt
metrics_df[['train_prec1', 'val_prec1']].plot(xlim=(0, len(metrics_df)), ylim=(0, 100))
metrics_df[['train_loss', 'val_loss']].plot(xlim=(0, len(metrics_df)))
plt.show()

In [ ]:
metrics_df

In [ ]:
metrics_df.loc[metrics_df['val_prec1'].idxmax()]